# PROJECT PRAKTIKUM PENGOLAHAN CITRA DIGITAL

In [2]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math

Pada code di atas, terdapat library-library yang digunakan untuk memudahkan program dalam menyelesaikan tugas tertentu. Pada library cv2 digunakan untuk pemrosesan gambar, library matplotlib.pyplot digunakan untuk membuat visualisasi data dalam bentuk grafik, library numpy digunakan untuk melakukan komputasi numerik dan manipulasi array multidimensi, library pandas digunakan untuk manipulasi dan analisis data, library math digunakan untuk melakukan operasi matematika, dan library os digunakan untuk mengakses dan memanipulasi direktori dan variabel serta menjalankan perintah sistem.

In [3]:
def process_dataset(dataset_dir):
    data = []
    labels = []
    paths = []

    for sub_folder in os.listdir(dataset_dir):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, sub_folder))
        for i, filename in enumerate(sub_folder_files):
            img_path = os.path.join(dataset_dir, sub_folder, filename)
            img = cv.imread(img_path)
            data.append(img)
            labels.append(sub_folder)
            paths.append(img_path)

    labels = np.array(labels)
    paths = np.array(paths)

    return data, labels, paths

Pada code di atas, terdapat function "process_dataset()" dengan parameter "dataset_dir". Function ini bertujuan untuk memproses dataset yang terdiri dari beberapa folder dengan masing-masing folder berisi gambar-gambar. Dimana function ini akan membaca gambar-gambar dalam dataset lalu menyimpannya ke dalam array data dan menyimpan label dari setiap gambar ke dalam array labels, dan menyimpan jalur lengkap dari setiap gambar ke dalam array paths.

In [16]:
def color_segmentation(images):
    segmented_images = []
    for image in images:
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
        lower_green = np.array([36, 40, 40])
        upper_green = np.array([70, 255, 255])
        mask = cv.inRange(hsv, lower_green, upper_green)
        segmented_image = cv.bitwise_and(image, image, mask=mask)
        segmented_images.append(segmented_image)
    return segmented_images

Pada code di atas, terdapat function "color_segmentation()" dengan parameter "images". Function ini bertujuan untuk melakukan segmentasi warna pada setiap gambar dalam daftar images dengan menggunakan space warna HSV. Lalu hasil segmentasi warna akan disimpan dalam daftar "segmented_images" dan dikembalikan sebagai output fungsi.

In [17]:
def edge_detection(images):
    edge_images = []
    for image in images:
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        edges = cv.Canny(gray, 50, 255)
        edge_images.append(edges)
    return edge_images

Pada code di atas, terdapat function "edge_detection()" dengan parameter "images". Function ini bertujuan untuk melakukan deteksi tepi pada setiap gambar dalam daftar "images" dengan mengkonversi gambar ke skala abu-abu dan menggunakan algoritma Canny. Dimana hasil deteksi tepi akan disimpan dalam daftar "edge_images" dan dikembalikan sebagai output fungsi.

In [18]:
def enhance_image(images):
    enhanced_images = []
    for image in images:
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        equalized_image = cv.equalizeHist(gray)
        enhanced_image = cv.cvtColor(equalized_image, cv.COLOR_GRAY2BGR)
        enhanced_images.append(enhanced_image)
    return enhanced_images

Pada code di atas, terdapat function "enhance_image()" dengan parameter "images". Function ini bertujuan untuk meningkatkan kualitas gambar pada setiap gambar dalam daftar "images" dengan menggunakan teknik equalization pada histogram gambar. Dimana hasil peningkatan kualitas gambar akan disimpan dalam daftar "enhanced_images" dan dikembalikan sebagai output fungsi.

In [19]:
def opening(images, kernel_size):
    opened_images = []
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    for image in images:
        eroded = cv.erode(image, kernel, iterations=1)
        opened = cv.dilate(eroded, kernel, iterations=1)
        opened_images.append(opened)
    return opened_images

Pada code di atas, terdapat function "opening()" dengan parameter "images" dan "kernel_size". Function ini bertujuan untuk melakukan operasi opening pada setiap gambar dalam daftar "images" dengan menggunakan operasi erosi dan dilasi. Dimana hasil operasi opening akan disimpan dalam daftar "opened_images" dan dikembalikan sebagai output fungsi.

In [20]:
def normalize_images(images):
    normalized_images = []
    for img in images:
        normalized_img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX)
        normalized_images.append(normalized_img)
    return normalized_images

Pada code di atas, terdapat function "normalize_images()" dengan parameter "images". Function ini bertujuan untuk melakukan normalisasi intensitas piksel pada setiap gambar dalam daftar "images" sehingga nilai pikselnya berada dalam rentang 0 hingga 255. Dimana hasil normalisasi disimpan dalam daftar "normalized_images" dan dikembalikan sebagai output fungsi.

In [21]:
dataset = "dataset/"
data, labels, paths = process_dataset(dataset)
segmented_images = color_segmentation(data)

enhanced_images = enhance_image(segmented_images)

normalized_images = normalize_images(enhanced_images)

opened_images = opening(normalized_images, kernel_size=2)

edge_images = edge_detection(opened_images)

Pada code di atas, dilakukan pendefinisian path direktori dataset yang akan digunakan. Lalu dilakukan pemanggilan function "process_dataset()" dengan argumen "dataset" untuk memproses dataset gambar. Kemudian terdapat pemanggilan function "color_segmentation()", "enhance_image()", "normalize_images()", "opening()", dan "edge_detection()" yang bertujuan untuk melakukan segmentasi warna, peningkatan kualitas gambar, normalisasi intensitas piksel, operasi opening, dan deteksi tepi yang disimpan pada variabel masing-masing. 

In [22]:
def classify_images(edge_images, paths):
    for edge_image, path in zip(edge_images, paths):
        file_name = os.path.basename(path)
        white_pixel_count = np.sum(edge_image > 0)
        threshold_dense = 5000  
        threshold_sparse = 1000
        if white_pixel_count > threshold_dense:
            classification = "hutan lebat"
        elif white_pixel_count < threshold_sparse:
            classification = "hutan gundul"
        else:
            classification = "setengah lebat"
        save_dir = os.path.join(dataset, classification)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, file_name)
        cv.imwrite(save_path, edge_image)

classify_images(edge_images, paths)


Pada code di atas, terdapat function "classify_images()" dengan parameter "edge_images" dan "paths". Function ini bertujuan untuk melakukan klasifikasi gambar berdasarkan jumlah piksel putih pada gambar hasil deteksi tepi. Gambar-gambar diklasifikasikan menjadi "hutan lebat", "hutan gundul", atau "setengah lebat" berdasarkan ambang batas yang ditentukan dan gambar hasil klasifikasi akan disimpan pada direktori yang sesuai dengan klasifikasinya.